In [ ]:
!pip install opencv-python


In [ ]:
 pip install mediapipe

In [ ]:
!pip install  scikit-learn==1.2.0


In [ ]:
import os
import cv2


DATA_DIR = './data'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

number_of_classes = 3
dataset_size = 100

cap = cv2.VideoCapture(0)
for j in range(number_of_classes):
    if not os.path.exists(os.path.join(DATA_DIR, str(j))):
        os.makedirs(os.path.join(DATA_DIR, str(j)))

    print('Collecting data for class {}'.format(j))

    done = False
    while True:
        ret, frame = cap.read()
        cv2.putText(frame, 'Ready? Press "Q" ! :)', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3,
                    cv2.LINE_AA)
        cv2.imshow('frame', frame)
        if cv2.waitKey(25) == ord('q'):
            break

    counter = 0
    while counter < dataset_size:
        ret, frame = cap.read()
        cv2.imshow('frame', frame)
        cv2.waitKey(25)
        cv2.imwrite(os.path.join(DATA_DIR, str(j), '{}.jpg'.format(counter)), frame)

        counter += 1

cap.release()
cv2.destroyAllWindows()


In [128]:
import os
import pickle

import mediapipe as mp
import cv2
import matplotlib.pyplot as plt

class Joint:
    def __init__(self, name,XValue, YValue):
     self.name = name
     self.XValue = XValue
     self.YValue = YValue

    def get_name(self):
     return self.name
    
class Finger:
    def __init__(self, name, joints):
        self.name = name
        self.joints = []  # Initialize joints as an empty list
        self.set_joints(joints)  # Call set_joints method to set the joints

    def get_name(self):
        return self.name

    def get_joints(self):
        # Check if any joint is None
        for joint in self.joints:
            if joint is None:
                return False
        return True

    def set_joints(self, joints):
        self.joints.extend(joints)

     
class Handie:
    def __init__(self, frame, fingers):
        self.frame = frame
        self.fingers = fingers

    def get_frame(self):
        return self.frame
    



mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

DATA_DIR = 'C:/Users/paulp/Desktop/Gestenerkennung/Datensatz_Kamera/Dur'



data = []
labels = []
frames = []
handies = []


Thumb = None
Index =  None
Middle =  None
Ring =  None
Pinky =  None

for dir_ in os.listdir(DATA_DIR):

    if dir_ == ".thumbs.db" or dir_ == "desktop.ini" :
        print("Skipping unwanted file in {dir_} directory: {img_path}")
        continue
    
    string_length = len(dir_)   
    substringdir_ = dir_[1: string_length] 

    full_dir_path = os.path.join(DATA_DIR,dir_)

   

    frame_count = 0
    framecounter=0
    jointcounter=0
    if full_dir_path == "C:/Users/paulp/Desktop/Gestenerkennung/Datensatz_Kamera/Dur\ADur_7c.mp4":
     print("checkvideostartup")
    


     cap=cv2.VideoCapture(full_dir_path)
     if(cap.isOpened()==False):
        print("couldnt open File")
        break
     while(cap.isOpened()):
      ret, frame = cap.read() 
      if ret == True: 
    # Display the resulting frame 
       if frame_count%2==0:
        print(frame_count)
        frames.append(frame_count)
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        temporaljoints=[]
        if results.multi_hand_landmarks:
          for hand_landmarks in results.multi_hand_landmarks:
             for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x* frame.shape[1]
                y = hand_landmarks.landmark[i].y*frame.shape[0]
                
                if i>=1 and i<=4:
                  print("Thumbvalue")
                  print(i)
                  joint=Joint("thumbjoint",x,y)
                  temporaljoints.append(joint)
                #create Thumbbone
                if  i==4:
                  Thumb=Finger("thumb", temporaljoints) 
                  temporaljoints=[] 
                if i>4 and i<=8:
                  print("Indexes")
                  print(i)
                  joint=Joint("indexjoint",x,y)
                  temporaljoints.append(joint)
                #createindexbone
                if i==8:
                    Index=Finger("index", temporaljoints)
                    temporaljoints=[]
                if i>8 and i<=12:
                  joint=Joint("middlejoint",x,y)
                  temporaljoints.append(joint)
                #createmiddlebone  
                if i==12:
                    Middle=Finger("middle", temporaljoints)
                    temporaljoints=[]   
                if i>12 and i<=16:
                  joint=Joint("ringjoint",x,y)
                  temporaljoints.append(joint)
                #createringbone  
                if i==12:
                    Ring=Finger("ring", temporaljoints)
                    temporaljoints=[]   
                if i>16 and i<=20:
                  joint=Joint("pinkyjoint",x,y)
                  temporaljoints.append(joint)
                  
                  if i==20:
                   Pinky=Finger("pinky", temporaljoints)
                   temporaljoints=[]   
                if(framecounter !=frame_count and i==jointcounter):
                 print("add hand")
                 jointcounter+=4
                 connectedjoints=[]
                 connectedjoints.append(Thumb)
                
                 connectedjoints.append(Index)
                 connectedjoints.append(Middle)
                 connectedjoints.append(Ring)
                 connectedjoints.append(Pinky)
                 currentHand=Handie(framecounter,connectedjoints)
                 handies.append(currentHand)
                 print("-------------------------")
                 print("createdHand")
                 framecounter=frame_count
                 currentHand=None
                 print(framecounter)     
        cv2.imshow('Image', frame)
       

       frame_count+=1
       delay = int(1000 / cap.get(cv2.CAP_PROP_FPS))  # Calculate delay based on video frame rate
       key = cv2.waitKey(delay)
  
       #count total number of frames and then close instances
       if frame_count ==int(cap.get(cv2.CAP_PROP_FRAME_COUNT)):
        print(frame_count)
        break
      print("Berechnung abgeschlossen hier sind die ergebnisse")

joint2Xvalues=[]
joint2Yvalues=[] 
print(handies[1].fingers[4].joints[5].XValue)
for handie in handies:
     joint2Xvalues.append(handie.fingers[0].joints[3].XValue)
     joint2Xvalues.append(handie.fingers[0].joints[1].XValue)
     joint2Xvalues.append(handie.fingers[0].joints[2].XValue)
     joint2Xvalues.append(handie.fingers[0].joints[3].XValue)
     joint2Yvalues.append(handie.fingers[0].joints[0].YValue)
     joint2Yvalues.append(handie.fingers[0].joints[1].YValue)
     joint2Yvalues.append(handie.fingers[0].joints[2].YValue)
     joint2Yvalues.append(handie.fingers[0].joints[3].YValue)


cap.release() 
      
      
   

   
       

   
    

# Closes all the frames 

     

#plt.scatter(range(len(x_)), x_)

# Add labels and title
#plt.xlabel(frames)
#plt.ylabel('Value')
#plt.title('Scatter Plot')

# Show plot
#plt.show()




           
         





    

checkvideostartup
0
Thumbvalue
1
Thumbvalue
2
Thumbvalue
3
Thumbvalue
4
Indexes
5
Indexes
6
Indexes
7
Indexes
8
Berechnung abgeschlossen hier sind die ergebnisse
Berechnung abgeschlossen hier sind die ergebnisse
2
add hand
2
Thumbvalue
1
Thumbvalue
2
Thumbvalue
3
Thumbvalue
4
Indexes
5
Indexes
6
Indexes
7
Indexes
8
Berechnung abgeschlossen hier sind die ergebnisse
Berechnung abgeschlossen hier sind die ergebnisse
4
Thumbvalue
1
Thumbvalue
2
Thumbvalue
3
Thumbvalue
4
add hand
4
Indexes
5
Indexes
6
Indexes
7
Indexes
8
Berechnung abgeschlossen hier sind die ergebnisse
Berechnung abgeschlossen hier sind die ergebnisse
6
Thumbvalue
1
Thumbvalue
2
Thumbvalue
3
Thumbvalue
4
Indexes
5
Indexes
6
Indexes
7
Indexes
8
add hand
6
Berechnung abgeschlossen hier sind die ergebnisse
Berechnung abgeschlossen hier sind die ergebnisse
8
Thumbvalue
1
Thumbvalue
2
Thumbvalue
3
Thumbvalue
4
Indexes
5
Indexes
6
Indexes
7
Indexes
8
add hand
8
Berechnung abgeschlossen hier sind die ergebnisse
Berechnung abgesch

In [ ]:
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np


data_dict = pickle.load(open('./data.pickle', 'rb'))

data = np.asarray(data_dict['data'])
print(data)
labels = np.asarray(data_dict['labels'])
print(labels)

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

model = RandomForestClassifier()

model.fit(x_train, y_train)

y_predict = model.predict(x_test)

score = accuracy_score(y_predict, y_test)

print('{}% of samples were classified correctly !'.format(score * 100))

f = open('model.p', 'wb')
pickle.dump({'model': model}, f)
f.close()


In [ ]:
import pickle

import cv2
import mediapipe as mp
import numpy as np

model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

labels_dict = {0: 'A', 1: 'B', 2: 'C'}
while True:

    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10

        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10

        prediction = model.predict([np.asarray(data_aux)])

        predicted_character = labels_dict[int(prediction[0])]
        if predicted_character!='A':
             print("anythingbut a")
        elif predicted_character =='A':
            print("Now its A")

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                    cv2.LINE_AA)

    cv2.imshow('frame', frame)
    cv2.waitKey(1)


cap.release()
cv2.destroyAllWindows()


In [ ]:
pip install scamp


In [ ]:
pip install python-rtmidi

In [ ]:
pip install pynput

In [ ]:
pip install scamp_extensions

In [ ]:
import pickle

import cv2
import mediapipe as mp
import numpy as np

from scamp import *

import time

s = Session()
to_sc= s.new_osc_part("sawtestone" , port=57120)

model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

labels_dict = {0: 'A', 1: 'B', 2: 'C'}


while True:

    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10

        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10

        prediction = model.predict([np.asarray(data_aux)])

        predicted_character = labels_dict[int(prediction[0])]
        if predicted_character!='A':
            print("Now its not A")
        elif predicted_character =='A':
            to_sc.play_note(36, 0.8 ,0.1)
            print("Now its A")
            time.sleep(2)
          
            

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                    cv2.LINE_AA)

    cv2.imshow('frame', frame)
    cv2.waitKey(1)


cap.release()
cv2.destroyAllWindows()
